## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
#! pip install gmaps
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Barrow,71.2906,-156.7887,-4.00,77,40,14.97,US,2021-01-18 02:48:25
1,1,Nikolskoye,59.7035,30.7861,19.40,93,75,6.71,RU,2021-01-18 02:51:15
2,2,Gedo,9.0167,37.4500,51.30,55,62,3.60,ET,2021-01-18 02:58:05
3,3,Rikitea,-23.1203,-134.9692,78.75,74,11,9.89,PF,2021-01-18 02:50:50
4,4,Oranjemund,-28.5500,16.4333,61.23,92,0,2.21,NaN,2021-01-18 02:58:05


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#ask the customer to add a minimum and maxiumum temperature value 
min_temp = float(input("What is the minimum temperature you would like for your trip"))
max_temp = float(input("What is the maximum temperature you would like for your tirp "))

What is the minimum temperature you would like for your trip60
What is the maximum temperature you would like for your tirp 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Rikitea,-23.1203,-134.9692,78.75,74,11,9.89,PF,2021-01-18 02:50:50
4,4,Oranjemund,-28.5500,16.4333,61.23,92,0,2.21,NaN,2021-01-18 02:58:05
7,7,Inhambane,-23.8650,35.3833,77.11,92,100,13.73,MZ,2021-01-18 02:58:06
8,8,New Norfolk,-42.7826,147.0587,69.80,43,40,25.32,AU,2021-01-18 02:58:06
9,9,Port Blair,11.6667,92.7500,82.24,72,49,19.55,IN,2021-01-18 02:58:07
15,15,Hobart,-42.8794,147.3294,69.80,43,40,25.32,AU,2021-01-18 02:50:43
16,16,Lagoa,39.0500,-27.9833,61.66,81,39,28.50,PT,2021-01-18 02:52:09
18,18,Georgetown,5.4112,100.3354,82.40,61,20,9.22,MY,2021-01-18 02:50:42
19,19,Ahipara,-35.1667,173.1667,68.99,57,7,10.85,NZ,2021-01-18 02:58:09
20,20,Butaritari,3.0707,172.7902,80.65,75,44,19.04,KI,2021-01-18 02:50:37


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       6
Date          0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(subset=["Country"])

In [8]:
clean_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp"
                     #, "Current Description"
                     , "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Rikitea,PF,78.75,-23.1203,-134.9692,
7,Inhambane,MZ,77.11,-23.8650,35.3833,
8,New Norfolk,AU,69.80,-42.7826,147.0587,
9,Port Blair,IN,82.24,11.6667,92.7500,
15,Hobart,AU,69.80,-42.8794,147.3294,
16,Lagoa,PT,61.66,39.0500,-27.9833,
18,Georgetown,MY,82.40,5.4112,100.3354,
19,Ahipara,NZ,68.99,-35.1667,173.1667,
20,Butaritari,KI,80.65,3.0707,172.7902,
23,Kapaa,US,78.80,22.0752,-159.3190,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"  
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make request and get the JSON data from the search.
    name_address = requests.get(base_url, params=params)
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    name_address = name_address.json()  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df   

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Rikitea,PF,78.75,-23.1203,-134.9692,Pension Maro'i
7,Inhambane,MZ,77.11,-23.8650,35.3833,Oceano
8,New Norfolk,AU,69.80,-42.7826,147.0587,The Shingles Riverside Cottages
9,Port Blair,IN,82.24,11.6667,92.7500,Welcomhotel Bay Island Port Blair
15,Hobart,AU,69.80,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
...,...,...,...,...,...,...
677,Namibe,AO,71.46,-15.1961,12.1522,Hotel Chik Chik Namibe
679,Pita,GN,66.18,10.8333,-12.5833,
680,Chicama,PE,71.44,-7.8447,-79.1469,Hospedaje Punta Cana PERU
682,Neiafu,TO,86.00,-18.6500,-173.9833,Mystic Sands


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City          0
Country       0
Max Temp      0
Lat           0
Lng           0
Hotel Name    0
dtype: int64

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#fig

In [15]:
marker_layer_2= gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer_2)
fig

Figure(layout=FigureLayout(height='420px'))